# Install the package

In [30]:
!python -m pip install pytorch-tabnet
!python -m pip install pytorch_lightning

You should consider upgrading via the 'c:\Users\lenovo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\lenovo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


# Use wandb

In [31]:
!wandb login cccce13d087a6622943e34e917c915d3f1c12906

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\lenovo\_netrc


In [1]:
import wandb

# Import with OS

In [2]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)



c:\Users\lenovo\OneDrive - Cong ty co phan Format Vietnam JSC\Desktop\ads_enfit


In [3]:
exps_dir = "../../exps"
if os.path.exists(exps_dir) == False: # tạo thư mục (nếu chưa có)
  os.makedirs(exps_dir, exist_ok=True)

save_dir = os.path.join(root_dir, 'exps/exp2')
os.makedirs(save_dir, exist_ok=True)

In [4]:
print(save_dir)

c:\Users\lenovo\OneDrive - Cong ty co phan Format Vietnam JSC\Desktop\ads_enfit\exps/exp2


# Import library/framework

In [5]:
from libs.common import *
from config.config import *

c:/Users/lenovo/OneDrive - Cong ty co phan Format Vietnam JSC/Desktop/ads_enfit


In [12]:
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torch.nn as nn
import torch.optim as optim
from pytorch_tabnet.pretraining import TabNetPretrainer


# Import data

In [7]:
X_train=pd.read_csv(f'{save_dir}/X_train.csv', index_col=None)
X_valid=pd.read_csv(f'{save_dir}/X_valid.csv', index_col=None)
y_train=pd.read_csv(f'{save_dir}/y_train.csv', index_col=None)
y_valid=pd.read_csv(f'{save_dir}/y_valid.csv', index_col=None)
X_test = pd.read_csv(f'{save_dir}/X_test.csv', index_col=None)
y_test = pd.read_csv(f'{save_dir}/y_test.csv', index_col=None)

In [8]:
X_train = X_train.drop(columns=["Unnamed: 0"])
X_valid = X_valid.drop(columns=["Unnamed: 0"])
y_train = y_train.drop(columns=["Unnamed: 0"])
y_valid = y_valid.drop(columns=["Unnamed: 0"])
X_test = X_test.drop(columns=["Unnamed: 0"])
y_test = y_test.drop(columns=["Unnamed: 0"])

In [9]:
class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

pretrain_dataset = TabularDataset(X_train, X_train)
train_dataset = TabularDataset(X_train, y_train)
valid_dataset = TabularDataset(X_valid, y_valid)
test_dataset = TabularDataset(X_test, y_test)

pretrain_loader = DataLoader(pretrain_dataset, batch_size=32, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataset = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Pretrain with basic AE

Take a long time to converge, move to the pretrain with Tabnet

In [39]:
class PretrainedModel(nn.Module):
    def __init__(self, input_dim):
        super(PretrainedModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.dropout3 = nn.Dropout(0.3)
        self.fc4 = nn.Linear(64, 128)
        self.bn4 = nn.BatchNorm1d(128)
        self.dropout4 = nn.Dropout(0.3)
        self.fc5 = nn.Linear(128, 256)
        self.bn5 = nn.BatchNorm1d(256)
        self.dropout5 = nn.Dropout(0.3)
        self.fc6 = nn.Linear(256, input_dim)  # Changed output dimension to match input dimension
        
    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = torch.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout4(x)
        x = torch.relu(self.bn5(self.fc5(x)))
        x = self.dropout5(x)
        x = self.fc6(x)
        return x

input_dim = X_train.shape[1]
model = PretrainedModel(input_dim)


In [41]:
import torch.nn.functional as F

def train_pretrain(model, device, train_loader, optimizer, epoch):
    model.to(device)
    #train_loader.dataset.to(device)
    model.train()
    total_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        optimizer.zero_grad()
        recon_batch = model(data)
        loss = F.mse_loss(recon_batch, data)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if (batch_idx + 1) % 50 == 0:
            print(f'Epoch {epoch+1}, Batch {batch_idx+1}, Loss: {total_loss / 50}')
            total_loss = 0
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}')

# Example usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# pretrain_loader.dataset.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    train_pretrain(model, device, pretrain_loader, optimizer, epoch)


Epoch 1, Batch 50, Loss: 6781848611.84
Epoch 1, Batch 100, Loss: 6804729610.24
Epoch 1, Batch 150, Loss: 6652121907.2
Epoch 1, Batch 200, Loss: 6851479162.88
Epoch 1, Batch 250, Loss: 6749243187.2
Epoch 1, Batch 300, Loss: 6706622443.52
Epoch 1, Batch 350, Loss: 6663213056.0
Epoch 1, Batch 400, Loss: 6733930905.6
Epoch 1, Batch 450, Loss: 6683670067.2
Epoch 1, Batch 500, Loss: 6756838748.16
Epoch 1, Batch 550, Loss: 6818785720.32
Epoch 1, Batch 600, Loss: 6900345098.24
Epoch 1, Batch 650, Loss: 6709935626.24
Epoch 1, Batch 700, Loss: 6629738956.8
Epoch 1, Batch 750, Loss: 6769588695.04
Epoch 1, Batch 800, Loss: 6847376936.96
Epoch 1, Batch 850, Loss: 6664008089.6
Epoch 1, Batch 900, Loss: 6657049999.36
Epoch 1, Batch 950, Loss: 6627681761.28
Epoch 1, Batch 1000, Loss: 6678654177.28
Epoch 1, Batch 1050, Loss: 6625803827.2
Epoch 1, Batch 1100, Loss: 6756674416.64
Epoch 1, Batch 1150, Loss: 6828769505.28
Epoch 1, Batch 1200, Loss: 6768721971.2
Epoch 1, Batch 1250, Loss: 6735470817.28
Epoc

KeyboardInterrupt: 

# Pretrain with Tabnet pretrain

In [26]:
unsupervised_model = TabNetPretrainer(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    mask_type='entmax', # "sparsemax"
    n_d=64, # Set the hidden_dim to 64
    n_a=64,
    device_name='cuda'
)

unsupervised_model.fit(
    X_train=X_train.to_numpy(),
    eval_set=[X_valid.to_numpy()],
    pretraining_ratio=0.8,
)

unsupervised_model.save_model('tabnet_pretrainer')

epoch 0  | loss: 169515.26133| val_0_unsup_loss_numpy: 480.5047607421875|  0:04:01s
epoch 1  | loss: 217.07925| val_0_unsup_loss_numpy: 447.44287109375|  0:08:38s
epoch 2  | loss: 205.98656| val_0_unsup_loss_numpy: 427.7922668457031|  0:13:03s
epoch 3  | loss: 207.85633| val_0_unsup_loss_numpy: 396.24853515625|  0:16:57s
epoch 4  | loss: 198.49121| val_0_unsup_loss_numpy: 399.6509704589844|  0:21:40s
epoch 5  | loss: 188.54674| val_0_unsup_loss_numpy: 367.6967468261719|  0:26:20s
epoch 6  | loss: 175.31442| val_0_unsup_loss_numpy: 357.1885986328125|  0:30:22s


KeyboardInterrupt: 

In [14]:
unsupervised_model = TabNetPretrainer()
unsupervised_model.load_model('tabnet_pretrainer.zip')

In [15]:
# pretrained_encoder = unsupervised_model.network.encoder

# # This is a torch module, use torch save to save this
# torch.save({'model_state_dict': pretrained_encoder.state_dict(), 'model_architecture': str(pretrained_encoder)}, 'tabnet_pretrainer_encoder.pth')

In [91]:
# unsupervised_model.network

TabNetPretraining(
  (embedder): EmbeddingGenerator()
  (masker): RandomObfuscator()
  (encoder): TabNetEncoder(
    (initial_bn): BatchNorm1d(141, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
    (initial_splitter): FeatTransformer(
      (shared): GLU_Block(
        (shared_layers): ModuleList(
          (0): Linear(in_features=141, out_features=32, bias=False)
          (1): Linear(in_features=16, out_features=32, bias=False)
        )
        (glu_layers): ModuleList(
          (0): GLU_Layer(
            (fc): Linear(in_features=141, out_features=32, bias=False)
            (bn): GBN(
              (bn): BatchNorm1d(32, eps=1e-05, momentum=0.02, affine=True, track_running_stats=True)
            )
          )
          (1): GLU_Layer(
            (fc): Linear(in_features=16, out_features=32, bias=False)
            (bn): GBN(
              (bn): BatchNorm1d(32, eps=1e-05, momentum=0.02, affine=True, track_running_stats=True)
            )
          )
        )


In [20]:
# Define the custom encoder class
class CustomTabNetEncoder(torch.nn.Module):
    def __init__(self, pretrainer_model):
        super(CustomTabNetEncoder, self).__init__()
        self.encoder = pretrainer_model.network.encoder

    def forward(self, x):
        # Pass data through the encoder to get the step outputs and feature importances
        steps_output, M_loss = self.encoder(x)
        final_step_output = steps_output[-1]
        return final_step_output, M_loss

# Initialize the encoder with the pre-trained model
encoder = CustomTabNetEncoder(unsupervised_model)
encoder.eval()  # Set the encoder to evaluation mode

# Move the encoder to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder.to(device)

# Create random tensor
random_data = torch.randn(10, 141)  # 10 samples, 141 features each

# Move the random tensor to the device
random_data_tensor = random_data.to(device).to(torch.float32)

# Encode the random data
with torch.no_grad():
    latent_representations, feature_importances = encoder(random_data_tensor)

# Print the outputs
print("Latent Representations:", latent_representations.shape)
print("Feature Importances (M_loss):", feature_importances)


Latent Representations: torch.Size([10, 8])
Feature Importances (M_loss): tensor(-0.4954)


In [22]:
# Test the first batch
print(f"First sample of random data: {random_data[0]}\n")
print(f"First sample in latent representations: {latent_representations[0]}")

First sample of random data: tensor([ 1.2233, -0.1336,  2.4271,  0.7167, -1.7371, -0.3371, -1.8067, -1.7551,
         1.9278,  0.6380, -1.3735, -0.1758, -0.4896, -0.8453, -0.1855, -1.1828,
        -0.7313, -0.1734,  1.0634, -1.1755, -0.3373,  1.5613,  0.3157,  0.3237,
        -1.5060, -0.1666,  0.2775, -0.3152,  0.4184, -0.3998,  0.9849, -0.4676,
        -0.8584, -1.8549,  0.1904,  0.2829, -0.6988,  0.0996, -0.8062, -0.0351,
        -0.3407,  1.6213,  0.7040, -0.1762, -0.3581, -0.1252, -1.0481, -0.7461,
        -2.1339, -2.3023,  0.3318,  0.6340, -0.6067, -2.2478,  0.8009, -0.2119,
         1.1917,  0.4351,  0.7897,  1.7413,  0.8892, -0.2036,  0.0980, -0.3547,
        -0.1185, -0.4851, -0.8448, -0.1437, -0.4034,  0.6374,  0.3527, -0.5134,
        -0.3074,  2.7839,  0.3000,  0.6985,  0.7986, -2.2435, -0.3787, -1.2733,
         0.7195, -1.5267, -0.1683,  0.5944,  1.1460,  2.5657,  0.6367, -0.0327,
        -0.3165, -0.1675,  1.7093,  0.9208,  0.5436,  1.1848,  0.1269,  0.5602,
         1.

In [93]:
import torch

# Example random data: 10 samples, 141 features each
random_data = torch.randn(10, 141)  # 10 samples, 20 features each

# Move the random tensor to the device
random_data_tensor = random_data.to(device).to(torch.float32)

# Encode the random data
with torch.no_grad():
    fc_output, feature_importances = encoder(random_data_tensor)

# Print the outputs
print("FC Output:", fc_output)
print("Feature Importances:", feature_importances)



FC Output: tensor([[0.1616, 0.0825, 0.5165, 0.0000, 0.3087, 1.5011, 1.6584, 0.0000],
        [0.1617, 0.0858, 0.4999, 0.0000, 0.3056, 1.5118, 1.6642, 0.0000],
        [0.1612, 0.0841, 0.5074, 0.0000, 0.3134, 1.5108, 1.6724, 0.0000],
        [0.1644, 0.0806, 0.5248, 0.0000, 0.3048, 1.4918, 1.6454, 0.0000],
        [0.0000, 1.2509, 0.0000, 4.8240, 0.0000, 3.1320, 0.0000, 0.0000],
        [2.9493, 0.0000, 0.0000, 0.0000, 0.0000, 2.9895, 5.4178, 3.5081],
        [0.1662, 0.0799, 0.5275, 0.0000, 0.3028, 1.4882, 1.6398, 0.0000],
        [0.0000, 0.1215, 0.5363, 0.4715, 0.3683, 1.3670, 0.9770, 0.0000],
        [0.1673, 0.0811, 0.5206, 0.0000, 0.3037, 1.4936, 1.6460, 0.0000],
        [0.0350, 0.0483, 0.7495, 0.0239, 0.3506, 1.3797, 1.4839, 0.0000]])
Feature Importances: tensor(-0.8036)


# Pretrain with VAE

In [76]:
import pytorch_lightning as pl

class VAE(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim * 2)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        x = self.encoder(x)
        mu, logvar = torch.chunk(x, 2, dim=-1)
        
        # Clamp logvar to prevent numerical instability
        logvar = torch.clamp(logvar, min=-10, max=2)  # Adjust as necessary
        
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

    def training_step(self, batch, batch_idx):
        x, _ = batch
        x_recon, mu, logvar = self.forward(x)

        # Diagnostics: Check for NaNs in logvar
        if torch.isnan(logvar).any():
            print("NaN detected in logvar")
        
        # Add a small value to logvar to prevent numerical issues
        logvar = torch.clamp(logvar, min=-10, max=10)
        
        recon_loss = nn.functional.mse_loss(x_recon, x, reduction='sum')
        kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        loss = recon_loss + kl_loss
        print(loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

vae_model = VAE(input_dim = X_test.shape[1], hidden_dim = 128, latent_dim = 64)

In [77]:
x = next(iter(train_loader))[0]  # Get the first batch of x
optimizer.zero_grad()  # Zero the gradients
output, _, _ = vae_model(x)  # Forward pass through the VAE model
print(output)
print(output.shape)  # Print the shape of the output

tensor([[ 1301.0930,  3075.4138,  7214.8564,  ...,  4246.9800,  2207.7947,
           467.3550],
        [ 1112.5470,  2620.4551,  6194.1484,  ...,  3626.0054,  1878.1315,
           392.9468],
        [ 1852.4546,  4418.6499, 10413.0918,  ...,  6097.6167,  3141.7329,
           657.8369],
        ...,
        [ 1781.8385,  4273.2471, 10055.7354,  ...,  5894.2725,  3029.8838,
           637.3226],
        [  603.0688,  1442.4968,  3378.1577,  ...,  1989.8705,  1032.0741,
           211.5147],
        [ 2217.4011,  5287.3174, 12433.1357,  ...,  7292.9141,  3765.7244,
           800.4923]], grad_fn=<AddmmBackward0>)
torch.Size([32, 141])


In [78]:
output,_,_ = vae_model(torch.randn(20, 141))

In [79]:
output

tensor([[ 0.2007, -0.1785, -0.1725,  ...,  0.1502, -0.3017,  0.2442],
        [ 0.0374, -0.4537,  0.0973,  ...,  0.2426, -0.3924,  0.3847],
        [ 0.1518,  0.5820, -0.1057,  ..., -0.1026,  0.1041,  0.4677],
        ...,
        [ 0.2070,  0.0596,  0.4023,  ...,  0.0873, -0.1273,  0.1061],
        [ 0.0619,  0.3165, -0.0754,  ...,  0.2179, -0.2738, -0.0085],
        [ 0.0036,  0.0472,  0.2800,  ...,  0.3336, -0.2945,  0.0604]],
       grad_fn=<AddmmBackward0>)

In [80]:
vae_model

VAE(
  (encoder): Sequential(
    (0): Linear(in_features=141, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=64, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=141, bias=True)
  )
)

In [82]:
# Initialize model, optimizer
input_dim = X_train.shape[1]
hidden_dim = 128
latent_dim = 64
vae_model = VAE(input_dim = X_test.shape[1], hidden_dim = 128, latent_dim = 64)
optimizer = torch.optim.Adam(vae_model.parameters(), lr=1e-3)
criterion = nn.MSELoss(reduction='sum')

# Training loop
vae_model.train()
num_epochs = 100
log_interval = 2500

for epoch in range(num_epochs):
    epoch_loss = 0
    for batch_idx, (x,y) in enumerate(train_loader):
        optimizer.zero_grad()  # Zero the gradients
        x_recon, _, _ = vae_model(x)  # Forward pass
        
        # Compute reconstruction loss
        recon_loss = criterion(x_recon, x)
        
        # Total loss
        loss = recon_loss #+ kl_loss
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
        # Log loss at intervals
        if batch_idx % log_interval == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item()}')

    print(f'Epoch {epoch + 1}, Average Loss: {epoch_loss / len(train_loader.dataset)}')

print("Training completed")

Epoch [1/100], Batch [0/33733], Loss: 23663749365760.0
Epoch [1/100], Batch [2500/33733], Loss: 437954304.0
Epoch [1/100], Batch [5000/33733], Loss: 598092544.0
Epoch [1/100], Batch [7500/33733], Loss: 239597344.0
Epoch [1/100], Batch [10000/33733], Loss: 8406275584.0
Epoch [1/100], Batch [12500/33733], Loss: 142291008.0
Epoch [1/100], Batch [15000/33733], Loss: 1895530112.0
Epoch [1/100], Batch [17500/33733], Loss: 787751232.0
Epoch [1/100], Batch [20000/33733], Loss: 174201504.0
Epoch [1/100], Batch [22500/33733], Loss: 119082104.0
Epoch [1/100], Batch [25000/33733], Loss: 500142464.0
Epoch [1/100], Batch [27500/33733], Loss: 773646912.0
Epoch [1/100], Batch [30000/33733], Loss: 282062720.0
Epoch [1/100], Batch [32500/33733], Loss: 94347912.0
Epoch 1, Average Loss: 518535074.2554273
Epoch [2/100], Batch [0/33733], Loss: 66117192.0
Epoch [2/100], Batch [2500/33733], Loss: 81347680.0
Epoch [2/100], Batch [5000/33733], Loss: 112149480.0
Epoch [2/100], Batch [7500/33733], Loss: 298460467

KeyboardInterrupt: 

In [ ]:
# Save the encoder
encoder = vae_model.encoder
torch.save(encoder.state_dict(), 'vae_encoder.pth')